# Fraud Detection with Feature Store on OpenShift AI 3.2

This notebook demonstrates the **full ML lifecycle** using **Feature Store** (Feast), **Model Registry**, and **Model Serving** (KServe/OVMS) on **Red Hat OpenShift AI 3.2** for real-time bank fraud detection.

## Prerequisites

- Workbench created in the `fraud-detection-ml` project
- Feature Store `fraud_detection` selected in the workbench configuration
- Data Connection to MinIO configured (for historical features and model storage)
- Model Registry `fraud-detection` available

## Workflow
1. Connect to the Feature Store (auto-mounted client config)
2. Explore registered features
3. Retrieve historical features for training
4. Train a fraud detection model (RandomForest)
5. Real-time prediction via the online store (local model)
6. **Export model to ONNX**
7. **Upload model to MinIO (S3)**
8. **Register in the Model Registry**
9. **Deploy for serving (KServe + OVMS)**
10. **End-to-end inference: Feast + Model Serving**

## 1. Connect to the Feature Store

The Feast client configuration is **auto-mounted** by RHOAI when you select the Feature Store in the workbench settings. The config file is at `/opt/app-root/src/feast-config/<project_name>`.

> **Note:** If you use the custom workbench image (`fraud-detection-datascience`), all dependencies are pre-installed. No `pip install` needed.

In [ ]:
# Only needed if NOT using the custom workbench image
# !pip install -q feast[postgres] scikit-learn pandas pyarrow s3fs boto3 skl2onnx onnxruntime model-registry

## 2. Connect to the Feature Store

In [ ]:
import os

from feast import FeatureStore

# The Feast client config is auto-mounted by RHOAI when you select
# the Feature Store in the workbench settings.
feast_config_dir = "/opt/app-root/src/feast-config"

if os.path.isdir(feast_config_dir):
    config_files = [
        os.path.join(feast_config_dir, f)
        for f in os.listdir(feast_config_dir)
        if os.path.isfile(os.path.join(feast_config_dir, f))
    ]
else:
    config_files = []

if config_files:
    fs_yaml = config_files[0]
    print(f"Using auto-mounted config: {fs_yaml}")
    with open(fs_yaml) as f:
        print(f.read())
    store = FeatureStore(fs_yaml_file=fs_yaml)
else:
    raise FileNotFoundError(
        f"No Feast config found in {feast_config_dir}. "
        "Make sure you selected the Feature Store when creating the workbench."
    )

print(f"\nProject: {store.project}")

## 3. Explore registered features

The features, entities, and feature views are already registered via `feast apply` (done during deployment).

In [ ]:
print("=== Entities ===")
for entity in store.list_entities():
    print(f"  - {entity.name}")

print("\n=== Feature Views ===")
for fv in store.list_feature_views():
    print(f"  - {fv.name} ({len(fv.features)} features, TTL={fv.ttl})")
    for feature in fv.features:
        print(f"      {feature.name}: {feature.dtype}")

print("\n=== On-Demand Feature Views ===")
for odfv in store.list_on_demand_feature_views():
    print(f"  - {odfv.name}")
    for feature in odfv.features:
        print(f"      {feature.name}: {feature.dtype}")

print("\n=== Data Sources ===")
for ds in store.list_data_sources():
    print(f"  - {ds.name} ({type(ds).__name__})")

## 4. Retrieve historical features (Training)

We retrieve features from the **offline store** (Parquet files on MinIO/S3) to train a fraud detection model.

This requires the MinIO Data Connection configured in the workbench (provides `AWS_*` environment variables).

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)

now = pd.Timestamp.now()
customer_ids = [f"C{str(i).zfill(5)}" for i in range(1, 51)]
N_TRANSACTIONS = 500

# Simulate labeled transaction data
entity_df = pd.DataFrame({
    "customer_id": np.random.choice(customer_ids, N_TRANSACTIONS),
    "event_timestamp": [now - pd.Timedelta(hours=np.random.randint(1, 24)) for _ in range(N_TRANSACTIONS)],
    "transaction_amount": np.round(np.random.exponential(200, N_TRANSACTIONS), 2),
    "is_foreign_transaction": np.random.choice([0, 1], N_TRANSACTIONS, p=[0.85, 0.15]),
})

# Features to retrieve from offline store (materialized feature views only)
offline_feature_refs = [
    "customer_profile:age",
    "customer_profile:account_age_days",
    "customer_profile:credit_limit",
    "customer_profile:num_cards",
    "transaction_stats:avg_transaction_amount_30d",
    "transaction_stats:num_transactions_7d",
    "transaction_stats:num_transactions_1d",
    "transaction_stats:max_transaction_amount_7d",
    "transaction_stats:num_foreign_transactions_30d",
    "transaction_stats:num_declined_transactions_7d",
]

# Retrieve historical features from the offline store (Parquet on S3)
print(f"Retrieving historical features from S3 ({N_TRANSACTIONS} rows)...")
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=offline_feature_refs,
).to_df()

# Compute on-demand features locally (avoids heavy Feast join for computed features)
training_df["amount_ratio_to_avg"] = (
    training_df["transaction_amount"] / training_df["avg_transaction_amount_30d"].clip(lower=1)
)
training_df["amount_ratio_to_max"] = (
    training_df["transaction_amount"] / training_df["max_transaction_amount_7d"].clip(lower=1)
)
training_df["risk_score"] = (
    training_df["amount_ratio_to_avg"] * 0.4
    + training_df["amount_ratio_to_max"] * 0.3
    + training_df["is_foreign_transaction"] * 0.3
)

# Full feature refs (for online store calls later)
feature_refs = offline_feature_refs + [
    "fraud_risk_features:amount_ratio_to_avg",
    "fraud_risk_features:amount_ratio_to_max",
    "fraud_risk_features:risk_score",
]

print(f"Training dataset: {training_df.shape[0]} rows, {training_df.shape[1]} columns")
training_df.head(10)

## 5. Train a fraud detection model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Generate simulated fraud labels
# Transactions with a high risk_score are more likely to be fraudulent
training_df = training_df.dropna()
fraud_probability = 1 / (1 + np.exp(-(training_df["risk_score"] - 1.5) * 3))
training_df["is_fraud"] = (np.random.random(len(training_df)) < fraud_probability).astype(int)

print(f"Fraud distribution:")
print(training_df["is_fraud"].value_counts())
print(f"Fraud rate: {training_df['is_fraud'].mean():.2%}")

# Prepare features for the model
model_features = [
    "age", "account_age_days", "credit_limit", "num_cards",
    "avg_transaction_amount_30d", "num_transactions_7d", "num_transactions_1d",
    "max_transaction_amount_7d", "num_foreign_transactions_30d",
    "num_declined_transactions_7d",
    "transaction_amount", "is_foreign_transaction",
    "amount_ratio_to_avg", "amount_ratio_to_max", "risk_score",
]

X = training_df[model_features]
y = training_df["is_fraud"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred, target_names=["Legitimate", "Fraud"]))

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Feature importance
import matplotlib.pyplot as plt

importances = pd.Series(clf.feature_importances_, index=model_features).sort_values(ascending=True)

fig, ax = plt.subplots(figsize=(10, 6))
importances.plot(kind="barh", ax=ax)
ax.set_title("Feature importance for fraud detection")
ax.set_xlabel("Importance")
plt.tight_layout()
plt.show()

## 6. Real-time prediction via the Online Store

Simulate an incoming transaction: retrieve the customer's features from the **online store** (PostgreSQL) in real-time, then apply the model.

In [ ]:
# Simulate a suspicious transaction
test_customer = "C00042"
transaction = {
    "transaction_amount": 4500.00,  # high amount
    "is_foreign_transaction": 1,     # foreign transaction
}

print(f"Incoming transaction for customer {test_customer}:")
print(f"  Amount: {transaction['transaction_amount']} EUR")
print(f"  Foreign transaction: {'Yes' if transaction['is_foreign_transaction'] else 'No'}")

# Retrieve features in real-time from PostgreSQL (online store)
online_features = store.get_online_features(
    entity_rows=[
        {
            "customer_id": test_customer,
            **transaction,
        }
    ],
    features=feature_refs,
).to_dict()

print("\nFeatures retrieved from the online store (PostgreSQL):")
for key, values in online_features.items():
    if key != "customer_id":
        print(f"  {key}: {values[0]}")

In [ ]:
# Build feature vector for prediction
feature_vector = {}
feature_vector.update(online_features)
feature_vector.update({k: [v] for k, v in transaction.items()})

predict_df = pd.DataFrame(feature_vector)

# Keep only the model features (in the right order)
available_features = [f for f in model_features if f in predict_df.columns]
predict_input = predict_df[available_features]

# Predict
prediction = clf.predict(predict_input)[0]
probability = clf.predict_proba(predict_input)[0]

print("\n" + "=" * 50)
if prediction == 1:
    print(f"FRAUD ALERT - Probability: {probability[1]:.1%}")
    print("Action: Transaction blocked for review")
else:
    print(f"Legitimate transaction - Fraud probability: {probability[1]:.1%}")
    print("Action: Transaction approved")
print("=" * 50)

## Architecture Summary

```
                        OpenShift AI 3.2
    +----------------------------------------------------------+
    |                                                          |
    |   +-------------+        +-------------------------+     |
    |   |  Workbench   |        |  Feature Store          |     |
    |   |  (Notebook)  |------->|  (Feast Operator)       |     |
    |   +------+------+        +----------+--------------+     |
    |          |                           |                    |
    |          |          +---------------+-----------+         |
    |          |          v               v           v         |
    |          |  +---------------+ +-----------+  +--------+  |
    |          |  | Offline Store | | Online    |  |   S3   |  |
    |          |  | (Parquet/S3)  | | Store     |  | (MinIO)|  |
    |          |  | Training      | | (Postgres)|  | Reg    |  |
    |          |  +---------------+ +-----------+  | +Data  |  |
    |          |                                   | +Models |  |
    |          v                                   +--------+  |
    |   +-------------+   +----------------+                   |
    |   |   Model      |   | Model Serving  |                   |
    |   |   Registry   |-->| (KServe/OVMS)  |                   |
    |   +-------------+   +----------------+                   |
    +----------------------------------------------------------+
```

## 8. Export the model to ONNX

We convert the trained RandomForest to ONNX format for serving with OpenVINO Model Server (OVMS).

In [ ]:
# Dependencies already installed in custom workbench image
# !pip install -q skl2onnx onnxruntime model-registry

In [ ]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as ort

# Convert sklearn model to ONNX
initial_type = [("input", FloatTensorType([None, len(model_features)]))]
onnx_model = convert_sklearn(clf, initial_types=initial_type)

# Save locally
onnx_path = "/tmp/fraud_model/1/model.onnx"
os.makedirs(os.path.dirname(onnx_path), exist_ok=True)

with open(onnx_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"Model exported to {onnx_path}")
print(f"Model size: {os.path.getsize(onnx_path) / 1024:.1f} KB")

# Quick validation with ONNX Runtime
session = ort.InferenceSession(onnx_path)
test_input = X_test.iloc[:3].values.astype(np.float32)
onnx_preds = session.run(None, {"input": test_input})
print(f"\nONNX validation - predictions: {onnx_preds[0].flatten()}")
print(f"Sklearn validation - predictions: {clf.predict(X_test.iloc[:3])}")
print("ONNX export validated successfully!")

## 9. Upload model to MinIO (S3)

Upload the ONNX model to the `models` bucket in MinIO. OVMS expects the model at `<model_name>/<version>/model.onnx`.

In [ ]:
import boto3

s3 = boto3.client(
    "s3",
    endpoint_url=os.environ["AWS_ENDPOINT_URL"],
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    region_name=os.environ.get("AWS_DEFAULT_REGION", "us-east-1"),
)

# Ensure bucket exists
try:
    s3.create_bucket(Bucket="models")
except s3.exceptions.BucketAlreadyOwnedByYou:
    pass
except Exception:
    pass  # bucket may already exist

# Upload: OVMS expects models/<model_name>/<version>/model.onnx
model_s3_key = "fraud-detection/1/model.onnx"
s3.upload_file(onnx_path, "models", model_s3_key)

model_uri = f"s3://models/{model_s3_key}"
print(f"Model uploaded to {model_uri}")

## 10. Register in the Model Registry

Register the model metadata in the RHOAI Model Registry. This stores the model name, version, S3 URI, format, and custom metadata (accuracy, features used, etc.).

In [ ]:
from model_registry import ModelRegistry
from sklearn.metrics import accuracy_score, f1_score

# Connect to the Model Registry
registry = ModelRegistry(
    server_address="http://fraud-detection.rhoai-model-registries.svc.cluster.local",
    port=8080,
    author="fraud-detection-notebook",
    is_secure=False,
)

# Compute metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Register the model
rm = registry.register_model(
    "fraud-detection",
    uri="s3://models/fraud-detection/1/model.onnx",
    version="1.0.0",
    description="Fraud detection model (RandomForest) trained with Feast features, exported to ONNX",
    model_format_name="onnx",
    model_format_version="1",
    storage_key="minio-data-connection",
    storage_path="fraud-detection/1/model.onnx",
    metadata={
        "accuracy": str(round(accuracy, 4)),
        "f1_score": str(round(f1, 4)),
        "framework": "sklearn",
        "n_features": str(len(model_features)),
        "features": ",".join(model_features),
        "feast_feature_views": "customer_profile,transaction_stats,fraud_risk_features",
    },
)

print(f"Model registered: {rm.name}")
print(f"  ID: {rm.id}")

# Verify
model = registry.get_registered_model("fraud-detection")
version = registry.get_model_version("fraud-detection", "1.0.0")
artifact = registry.get_model_artifact("fraud-detection", "1.0.0")
print(f"\nVerification:")
print(f"  Registered Model: {model.name} (ID: {model.id})")
print(f"  Version: {version.name} (ID: {version.id})")
print(f"  Artifact URI: {artifact.uri}")
print(f"  Format: {artifact.model_format_name}")

## 11. Deploy the model for serving (KServe + OVMS)

Create an `InferenceService` that deploys the ONNX model from MinIO using OpenVINO Model Server. The model will be served via a REST/gRPC endpoint.

In [ ]:
import subprocess
import json
import time

NAMESPACE = "fraud-detection-ml"

# Get model registry IDs for traceability labels
registered_model_id = model.id
model_version_id = version.id

# Create the InferenceService YAML
isvc_yaml = f"""
apiVersion: serving.kserve.io/v1beta1
kind: InferenceService
metadata:
  name: fraud-detection
  namespace: {NAMESPACE}
  labels:
    opendatahub.io/dashboard: "true"
    modelregistry/registered-model-id: "{registered_model_id}"
    modelregistry/model-version-id: "{model_version_id}"
  annotations:
    serving.kserve.io/deploymentMode: RawDeployment
    openshift.io/display-name: "Fraud Detection (ONNX)"
spec:
  predictor:
    model:
      modelFormat:
        name: onnx
        version: "1"
      runtime: kserve-ovms
      storageUri: "s3://models/fraud-detection"
      storage:
        key: minio-data-connection
        path: fraud-detection
      resources:
        requests:
          cpu: 500m
          memory: 512Mi
        limits:
          cpu: "1"
          memory: 1Gi
"""

# Write and apply
with open("/tmp/isvc.yaml", "w") as f:
    f.write(isvc_yaml)

result = subprocess.run(["oc", "apply", "-f", "/tmp/isvc.yaml"], capture_output=True, text=True)
print(result.stdout)
if result.returncode != 0:
    print(f"Error: {result.stderr}")

# Wait for the InferenceService to be ready
print("Waiting for InferenceService to be ready...")
for i in range(30):
    result = subprocess.run(
        ["oc", "get", "inferenceservice", "fraud-detection", "-n", NAMESPACE,
         "-o", "jsonpath={.status.conditions[?(@.type=='Ready')].status}"],
        capture_output=True, text=True
    )
    if result.stdout == "True":
        print("InferenceService is Ready!")
        break
    print(f"  Waiting... ({i+1}/30)")
    time.sleep(10)

# Show status
subprocess.run(["oc", "get", "inferenceservice", "-n", NAMESPACE], capture_output=False)

## 12. End-to-end inference: Feast online features + Model Serving

This is the full production flow:
1. Receive a transaction (customer ID + transaction details)
2. Fetch real-time features from Feast (online store)
3. Send the feature vector to the OVMS endpoint
4. Get the fraud prediction

In [ ]:
import requests

# Get the inference endpoint
result = subprocess.run(
    ["oc", "get", "inferenceservice", "fraud-detection", "-n", NAMESPACE,
     "-o", "jsonpath={.status.url}"],
    capture_output=True, text=True
)
inference_url = result.stdout.strip()

# For RawDeployment, use the service URL directly
if not inference_url:
    inference_url = f"http://fraud-detection-predictor.{NAMESPACE}.svc.cluster.local:8888"

print(f"Inference endpoint: {inference_url}")

# --- Simulate a suspicious transaction ---
test_customer = "C00003"
test_transaction = {
    "transaction_amount": 8500.00,
    "is_foreign_transaction": 1,
}

print(f"\nIncoming transaction for customer {test_customer}:")
print(f"  Amount: {test_transaction['transaction_amount']} EUR")
print(f"  Foreign: {'Yes' if test_transaction['is_foreign_transaction'] else 'No'}")

# Step 1: Fetch online features from Feast
online_features = store.get_online_features(
    entity_rows=[{"customer_id": test_customer, **test_transaction}],
    features=feature_refs,
).to_dict()

# Step 2: Build the feature vector in the correct order
feature_values = []
for feat in model_features:
    if feat in online_features:
        feature_values.append(float(online_features[feat][0] or 0))
    elif feat in test_transaction:
        feature_values.append(float(test_transaction[feat]))
    else:
        feature_values.append(0.0)

print(f"\nFeature vector ({len(feature_values)} features): {[round(v, 2) for v in feature_values]}")

# Step 3: Call the OVMS endpoint (V2 inference protocol)
payload = {
    "inputs": [
        {
            "name": "input",
            "shape": [1, len(model_features)],
            "datatype": "FP32",
            "data": feature_values,
        }
    ]
}

response = requests.post(
    f"{inference_url}/v2/models/fraud-detection/infer",
    json=payload,
    timeout=10,
)

print(f"\nResponse status: {response.status_code}")
result = response.json()

# Step 4: Parse the prediction
predictions = result["outputs"]
label_output = next((o for o in predictions if o["name"] == "output_label"), None)
proba_output = next((o for o in predictions if o["name"] == "output_probability"), None)

if label_output:
    is_fraud = label_output["data"][0]
    print(f"\n{'='*50}")
    if is_fraud == 1:
        print(f"FRAUD ALERT!")
    else:
        print(f"Legitimate transaction")
    if proba_output:
        probas = proba_output["data"]
        # probas is a flat list: [p_class0, p_class1] for each sample
        fraud_prob = probas[1] if len(probas) > 1 else probas[0]
        print(f"Fraud probability: {fraud_prob:.1%}")
    print(f"{'='*50}")
else:
    print(f"Raw output: {json.dumps(result, indent=2)}")